# Market Entry Analysis for Hawkeye Liquor Distributors

#### Objective:
Our goal with this dataset is to perform a multi-level market analysis for Hawkeye Liquor Distributors (fictional) using SQL, with a focus on facilitating the following business decisions:

1. __Location within Iowa__ - We will analyze which regions of Iowa have the highest/lowest sales, and make a recommendation on which areas would have the highest demand.

2. __Type of Liquor__ - Based on the regional sales of different types of liquor, we will make an overall recommendation on the style(s) of liquor Hawkeye should distribute to the State of Iowa.

3. __Pricing__ - For our recommended style(s) of liquor, we will advise Hawkeye on a fair price for distribution to the State of Iowa.

4. __Distribution Channel Mix__ - Finally, we will determine which type of store (grocery, wholesale, convenience, etc.) will yield the highest sales for Hawkeye's liquor products.

#### Data Brief: 
This dataset contains every wholesale purchase of liquor in the State of Iowa by retailers for sale to individuals since January 1, 2012. The State of Iowa controls the wholesale distribution of liquor intended for retail sale (off-premises consumption), which means this dataset offers a complete view of retail liquor consumption in the entire state. The dataset contains every wholesale order of liquor by all grocery stores, liquor stores, convenience stores, etc., with details about the store and location, the exact liquor brand and size, and the number of bottles ordered.

The original dataset comprises 24,229,431 records (null and non-null) and occupies 6.4 GB of storage as of October 3, 2022.

In order to stop the ingest of new data into the table for our analysis, we froze the raw dataset in a table called `ba775-a02-fall22.main.sales` in our BigQuery project on October 3, 2022.  Our analysis is based on this extract of the original dataset cited above.

#### Data sourced from Iowa Department of Commerce via Google BigQuery.  See __[here](https://data.iowa.gov/Sales-Distribution/Iowa-Liquor-Sales/m3tr-qhgy/data)__.

<hr />

## Table of Contents

#### I. Data Exploration

#### II. Data Cleaning

#### III. Data Scoping

#### IV. Conclusions

#### V. References

<hr />

## I. Data Exploration

#### Findings related to Decision 1, Location within Iowa:

__Q__: How is location represented in the data schema?

__A__: The original dataset includes location information in the `address`, `city`, `zip_code`, `store_location`, and `county` fields, which are all stored as STRING.  For the purposes of our analysis we will use the information from `county`.

__Q__: How many counties have liquor distributed from the original dataset?

__A__: 104 counties

In [ ]:
# %%bigquery
# SELECT 
#   COUNT(DISTINCT LOWER(county)) AS counties
# FROM `ba775-a02-fall22.main.sales`

__Q__: Which of those counties had the highest number of liquor sales?

__A__: Polk County

In [ ]:
# %%bigquery
# SELECT 
#     county, 
#     COUNT(invoice_and_item_number) AS value_occurrences
# FROM `ba775-a02-fall22.main.sales`
# GROUP BY county
# ORDER BY value_occurrences DESC
# LIMIT 1;

#### Findings related to Decision 2, Type of Liquor:

__Q__: How is type of liquor represented in the data schema?

__A__: Type of liquor information is stored in the `category` and `category_name` fields, which are stored as STRING.  We will use `category_name` for our analysis (see Data Cleaning for details).

__Q__: How many types of liquor were sold in the original dataset?

__A__: 112, excluding 1 NULL `category_name`

In [ ]:
# %%bigquery
# SELECT 
#   DISTINCT LOWER(category_name) AS lower_name,
# FROM `ba775-a02-fall22.main.sales`
# GROUP BY lower_name;

__Q__: Which type of liquor sold the most in the original dataset?

__A__: Canadian Whiskies

In [ ]:
# %%bigquery
# SELECT 
#   LOWER(category_name) AS lower_name,
#   COUNT(invoice_and_item_number) AS value_occurrences
# FROM `ba775-a02-fall22.main.sales`
# GROUP BY lower_name
# ORDER BY value_occurrences DESC
# LIMIT 1;

#### Findings related to Decision , Pricing:

__Q__: How is pricing information represented in the dataset?

__A__: The target field for our analysis is `state_bottle_cost` (FLOAT) since this price is what the Iowa Alcoholic Beverages Commission pays per bottle.  The `state_bottle_retail` (FLOAT) field contains the price each authorized vendor paid to the state for each bottle.

#### Findings related to Decision 5, Distribution Channel Mix:

__Q__: How is store information represented in the dataset?

__A__: Stores are identified by `store_name` and `store_number` fields as STRING in the original dataset.  In order to glean information regarding distribution channel mix, we will need to further categorize stores by their types (see Section II).

<hr />

## II. Data Cleaning

#### Data problems related to Decision 1, Location within Iowa:

__Q__: Is there missing information for location?  How will you address it?

__A__: No records are missing `county` information, therefore we will scope our analysis around information at the county-level.

__Q__: Were there other formatting or missing data issues?  What was their impact?

__A1__: Some `zip_code` records were stored as FLOAT causing multiple DISTINCT records for the same store.  Our overall analysis is not impacted by this error though, considering we are focused on county-level sales.

__A2__: Many counties had a mix of uppercase and lowercase naming conventions within their occurences in the data. We performed the conversion of all county name to their capitalized versions to overcome this issue.

In [ ]:
# %%bigquery
# UPDATE `ba775-a02-fall22.main.sales`
# SET county = UPPER(county)
# WHERE county != UPPER(county);

#### Data problems related to Decision 2, Type of Liquor:

__Q__: Is there missing information for type of liquor?  How will you address it?

__A__: 24,644 records for `category_name` are NULL.  However, considering this is 0.1% of the dataset, we will exclude these null records in our `main` table.

In [ ]:
# %%bigquery
# SELECT COUNT(*) 
# FROM `ba775-a02-fall22.main.sales`
# WHERE category_name IS NULL;

__Q__: Were there other formatting or missing data issues?  What was their impact?

__A1__: Performing COUNT DISTINCT of `category` shows 167 different categories of liquor due to 111 category numbers ending in ".0" as if a previous schema for the dataset stored this field as FLOAT.  This informed our decision to used `category_name` as the primary field for determining type of liquor.  

In [ ]:
# %%bigquery
# SELECT 
#   COUNT(DISTINCT category)
# FROM ba775-a02-fall22.main.sales
# WHERE category LIKE "%.0";

__A2__: Additionally, there are duplicate `category_name` values due to capitalized versions of values, which we fix using the below query 

In [ ]:
# %%bigquery
# UPDATE `ba775-a02-fall22.main.sales`
# SET category_name = UPPER(category_name)
# WHERE category_name != UPPER(category_name)

#### Data problems related to Decision 3, Pricing:

There are no significant data problems impacting our analysis related to pricing.

#### Data problems related to Decision 4, Distribution Channel Mix:

__Q__: Is there missing information on stores?  How will you address it?

__A__: No information is missing from `store_name`, however no clear categorization of stores exists in the original dataset.

<hr />

## III. Data Scoping

### Generate Revenue column

By multiplying columns `pack` and `state_bottle_cost` we can calculate revenue for each sale of liquor.  This key discovery from our data exploration provides us a fundamental business metric by which we can recommend the highest revenue decisions for Hawkeye Liquor Distributors (__see Query after next question__).

### How did you narrow down the data set for the purpose of your analysis?

We further dropped the following columns based on relevancy to the questions that we are trying to answer:

`invoice_and_item_number` - this unique identifier for all sales of liquor was duplicative given the other unique information about each record
    
`store_number` - duplicative and less informative than `store_name`

`item_number` - duplicative and less informative than `item_description`

`address`, `city`, `county_number` - too granular for the purpose of our analysis

`category` - duplicative with `category_name` and contains errors

`vendor_number` - duplicative with `vendor_name`

`sale_dollars` - refers to the sale of liquor from the store to an individual which is outside our scope

`volume_sold_liters`, `volume_sold_gallons` - easily calculated with `bottle_volume_ml` and other numeric data

In [ ]:
# %%bigquery
# CREATE OR REPLACE TABLE ba775-a02-fall22.main.sales_columns_dropped
# AS
#     (SELECT * EXCEPT
#         (invoice_and_item_number, 
#          store_number, 
#          item_number, 
#          address,
#          city,
#          county_number, 
#          category,
#          vendor_number, 
#          sale_dollars,
#          volume_sold_liters, 
#          volume_sold_gallons),
#     (pack * state_bottle_cost) AS revenue
# FROM `ba775-a02-fall22.main.sales` 

### Creating Bucketing for Prices

We create a new table `price_buckets` which marks the `state_bottle_cost` according to 8 different price buckets in multiples of $5. This is done to facilitate providing a focused pricing recommendation for Hawkeye

In [ ]:
# %%bigquery
# CREATE TABLE ba775-a02-fall22.main.price_buckets AS
# SELECT
# DISTINCT state_bottle_cost,
# CASE
#     WHEN state_bottle_cost > 75 THEN '75+'
#     WHEN state_bottle_cost BETWEEN 50 AND 75 THEN '50-75'
#     WHEN state_bottle_cost BETWEEN 25 AND 50 THEN '25-50'
#     WHEN state_bottle_cost BETWEEN 20 AND 25 THEN '20-25'
#     WHEN state_bottle_cost BETWEEN 15 AND 20 THEN '15-20'
#     WHEN state_bottle_cost BETWEEN 10 AND 15 THEN '10-15'
#     WHEN state_bottle_cost BETWEEN 5 AND 10 THEN '5-10'
#     WHEN state_bottle_cost BETWEEN 0 AND 5 THEN '0-5'
# END AS price_bucket
# FROM `ba775-a02-fall22.main.sales_columns_dropped`

### Creating Mapping for Distribution Channels

Within the Sales data, no clear categorization of stores exists in the original dataset, so we manually assigned `store_category` to all 2,424 stores in a new table called `store_to_channel_mapping` which we will JOIN later with our `sales` table.  The categories include: 'Convenience', 'Grocery', 'Hospitality', 'Liquor', and 'Wholesale'.  This categorization will inform our recommendation to Hawkeye Liquor Distributors on how to balance its approach to selling in a variety of store channels.

In [ ]:
# %%bigquery
# SELECT *
# FROM `ba775-a02-fall22.main.store_to_channel_mapping`
# LIMIT 100;

<hr />

## IV. Conclusions

### Location within Iowa

We saw earlier that Polk County has the highest number of liquor sales in Iowa. Upon analysing the highest revenue counties, we still see __Polk__ as the leader in sales with over __$467 Million__ in revenue. Therefore, we recommend Hawkeye Liquor Distributors to pursue their operations within Polk County.

<img src="https://i.ibb.co/hM8dVwg/Screenshot-2022-10-06-at-1-09-16-AM.jpg" width= 350 alt="Alt text that describes the graphic" title="Title text" />


In [ ]:
# %%bigquery
# SELECT 
#     UPPER(county), 
#     SUM(revenue) AS revenue
# FROM `ba775-a02-fall22.main.sales_columns_dropped`
# GROUP BY UPPER(county)
# ORDER BY revenue DESC
# LIMIT 10;

### Type of Liquor

__Canadian Whiskies__ have the highest sales in Polk County with __$38 Million__ total sales.  Therefore, Hawkeye Liquours should produce Canadian Whiskies if it seeks to target the highest demand liquor product in the Polk County, Iowa market.

<img src="https://i.ibb.co/WyphWWh/Screenshot-2022-10-06-at-1-06-36-AM.jpg" width= 350 alt="Alt text that describes the graphic" title="Title text" />

In [ ]:
# %%bigquery
# SELECT 
#     category_name,
#     SUM(revenue) AS revenue
# FROM
#     `ba775-a02-fall22.main.sales`
# WHERE 
#     UPPER(county) = 'POLK'
# GROUP BY category_name
# ORDER BY revenue DESC;

### Pricing

Joining the `sales` and `price_bucket` tables and grouping by the price buckets, we find the total revenue in each price bucket. For the Polk county and Canadian Whiskies category, We observe that the most revenue generating price range is the following:

##### \$5-10

Hence Hawkeye Liquours should price their Canadian Whiskies in the above range to generate maximum revenue.

<img src="https://i.ibb.co/kK3nsgK/Screenshot-2022-10-06-at-1-22-14-AM.jpg" width= 350 alt="Alt text that describes the graphic" title="Title text" />

In [ ]:
# %%bigquery
# SELECT
#    price_bucket, SUM(revenue) AS revenue
# FROM
#    (
#    SELECT *
#    FROM `ba775-a02-fall22.main.sales_columns_dropped`
#    WHERE UPPER(county) = 'POLK' AND category_name = 'CANADIAN WHISKIES'
#    )
# LEFT JOIN  
#    ba775-a02-fall22.main.price_buckets
# USING(state_bottle_cost)
# GROUP BY
#    price_bucket
# ORDER BY
#    revenue DESC;

### Distribution Channel

Joining the `sales`, `price_bucket` and `store_to_channel_mapping` tables and grouping by `store_category`, we find the total revenue in each distribution channel. For the Polk county and Canadian Whiskies category within the $5-10 price range, we observe that the most revenue generating distribution channels were the following:

##### Convenience
##### Grocery
##### Liquor

The above comprise of __95.5%__ of the total revenue from Canadian Whiskies sales (priced between $5-10) in Polk County, and therefore should be the 3 distribution channels on which Hawkeye should focus.

<img src="https://i.ibb.co/v1WNQBy/Screenshot-2022-10-06-at-1-26-23-AM.jpg" width= 350 alt="Alt text that describes the graphic" title="Title text" />

In [ ]:
# %%bigquery
# SELECT 
#     store_category, SUM(revenue) revenue
# FROM
#     (SELECT *
#     FROM ba775-a02-fall22.main.sales_columns_dropped
#     LEFT JOIN  
#     ba775-a02-fall22.main.price_buckets
#     USING(state_bottle_cost))
# INNER JOIN
#     ba775-a02-fall22.main.store_to_channel_mapping
# USING(store_name)
# WHERE
#     category_name = 'CANADIAN WHISKIES'
#     AND UPPER(county) = 'POLK'
#     AND price_bucket = '5-10'
# GROUP BY
#     store_category
# ORDER BY
#     revenue DESC

<hr />

## V. References

Iowa Department of Commerce. (2022). Iowa Liquor Sales. *Iowa Data*. https://data.iowa.gov/Sales-Distribution/Iowa-Liquor-Sales/m3tr-qhgy/data